<a href="https://colab.research.google.com/github/Praveen76/Introduction-to-LangChain-Chains/blob/main/Introduction-to-LangChain-Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **LangChain 🦜🔗 : Chains**

### **Objectives:**
At the end of the experiment you will be able to understand & use :

1. LLMChain
2. Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
3. Router Chain

#### **Installing and importing packages**

In [17]:
!pip install openai
!pip install langchain langchain-openai

In [18]:
import openai
import os
import pandas as pd

#### **Authentication key for OpenAI API**

In [7]:
import yaml
import json

from google.colab import drive

# Mount Google Drive with force remount
drive.mount('/content/drive', force_remount=True)

# Load relevant API Keys
file_path = '/content/drive/MyDrive/.API_KEYS/API_KEYS.yml'

with open(file_path, 'r') as file:
    api_keys = yaml.safe_load(file)


# Extract openai username and key
openai_key = api_keys['OPEN_AI']['Key']

os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key= os.getenv('OPENAI_API_KEY')

Mounted at /content/drive


#### **Model definition and loading the data**

In [8]:
llm_model = "gpt-3.5-turbo" # This is a chat model

#### **LLMChain** : Most Basic chain

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [10]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x78f5b0feea70>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x78f5b10141c0>, temperature=0.9, openai_api_key=SecretStr('**********'), openai_proxy='')

In [11]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [19]:
#chain = prompt|llm # LCEL --> LangChain Expression Language; Operation is from left to right i.e. write prompt first and then pass to LLM
chain = LLMChain(llm=llm, prompt=prompt) # Operation is from right to left i.e. write prompt first and then pass to LLM

In [20]:
product = "bed sheet"
result= chain.invoke(product)
result

{'product': 'bed sheet', 'text': '"CozyComfort Bedding"'}

LCEL(LangChain Expression Language)--> uses pipes(|):  When we call invoke() on the chain, the input is first passed to promp, whose output is then passed to the llm.

It's just the combination of the LLM
and the prompt. But now this chain will let us run through the
prompt and into the LLM in a sequential manner.

#### **Two types of sequential chains:**

  1. SimpleSequentialChain:Single input/output
  
  2. SequentialChain:Multiple inputs/outputs

##### **SimpleSequentialChain**

Sequential chain is another type of chains. The idea is to combine multiple chains where the output of the one chain is the input of the next chain. Each chain has just one input and one output





In [14]:
from langchain.chains import SimpleSequentialChain

In [15]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [16]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [21]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

In [22]:
product = "Electric Bike"
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
"VoltCycle"
VoltCycle offers an eco-friendly alternative to traditional transportation with their sleek and innovative electric bikes, perfect for urban commuting.

> Finished chain.


{'input': 'Electric Bike',
 'output': 'VoltCycle offers an eco-friendly alternative to traditional transportation with their sleek and innovative electric bikes, perfect for urban commuting.'}

#### **SequentialChain**

In [23]:
from langchain.chains import SequentialChain

In [24]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="English_Review"
                    )

In [25]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                    )

In [26]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [27]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [28]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","language", "followup_message"],
    verbose=True
)

In [29]:
review = """Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...
Vieux lot ou contrefaçon !?"""

In [30]:
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste poor. The foam does not hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer believes that the taste of the product is poor, the foam does not hold well, and suspects that they may have received an old batch or counterfeit product.',
 'language': 'French',
 'followup_message': "Bonjour, \n\nJe suis désolé(e) d'apprendre que vous avez eu une expérience décevante avec notre produit. Nous prenons votre feedback très au sérieux et nous voulons nous assurer que nos clients reçoivent toujours des produits de la meilleure qualité. Afin d'enquêter sur la situation, pourriez-vous nous donner plus d'informations sur votre achat, tel que la date d'achat et le lieu où vous l'avez acheté? Nous voulons nou

#### **Router Chain**

In [31]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

In [32]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

In [33]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [34]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [35]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [36]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [37]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [38]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [39]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [40]:
chain.invoke("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


{'input': 'What is black body radiation?',
 'text': "Black body radiation is the electromagnetic radiation emitted by a perfect absorber of radiation, known as a black body. A black body absorbs all incoming radiation and emits radiation across the entire electromagnetic spectrum. The spectrum of black body radiation is continuous and follows a specific distribution known as Planck's law. This phenomenon is important in understanding the behavior of objects at different temperatures and is a key concept in the field of thermal physics."}

In [41]:
chain.invoke("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


{'input': 'what is 2 + 2', 'text': '2 + 2 is equal to 4.'}

In [44]:
chain.invoke("What is jail breaking?")



> Entering new MultiPromptChain chain...
computer science: {'input': 'What is jailbreaking in terms of computer science?'}
> Finished chain.


{'input': 'What is jailbreaking in terms of computer science?',
 'text': "Jailbreaking in terms of computer science refers to the process of removing restrictions imposed by the manufacturer or operating system on a device, typically a smartphone or tablet. This allows users to gain access to features and functionalities that are not normally available, such as installing unauthorized apps, customizing the user interface, and bypassing digital rights management (DRM) restrictions. Jailbreaking involves exploiting vulnerabilities in the device's operating system to gain root access and modify system files. While jailbreaking can provide users with more control over their devices, it can also void warranties, compromise security, and potentially lead to instability or malfunctions."}